In [67]:
# File management
import os 

# Image loading and preprocessing
import keras.preprocessing as preprocess  

# CNN architecture and components
from keras.models import Sequential  
from keras.layers import (
    Conv2D,  
    Activation, 
    MaxPooling2D, 
    AveragePooling2D,
    Flatten,  
    Dense,  # For fully connected layers
    BatchNormalization,  
)

from keras.regularizers import l2
from keras.optimizers import Adam  
from keras.losses import CategoricalCrossentropy  

# Data manipulation and analysis
import pandas as pd  
import numpy as np  

# Keras utilities
import keras.utils as kutils  # helper functions for model building and evaluation

# Performance metrics
import sklearn.metrics as scores  

# Progress Bar
from tqdm import tqdm
from tensorflow.keras.callbacks import Callback

# Visualization
import plotly.graph_objects as go  
import plotly.offline as pyo  

In [68]:
hyper_parameters = {
    "batch-size": 8,
    "number-of-epochs": 100,
    'initial-learning-rate': 0.001,
    'l2-regularization': 0.001
}

In [69]:
#Retrieve eligible dataset directories
dataset_dirs = os.listdir("./dataset/")
eligible_dirs = ["./dataset/"+d for d in dataset_dirs]
eligible_dirs

['./dataset/B01P00',
 './dataset/B02P00',
 './dataset/B03P00',
 './dataset/B04P00',
 './dataset/B05P00',
 './dataset/B06P00',
 './dataset/B07P00',
 './dataset/B08P00',
 './dataset/B09P00',
 './dataset/B10P00']

In [70]:
# Collect image filenames from eligible directories
images = {}
for directory in eligible_dirs:
    images[directory] = next(os.walk(directory))[2]
images

{'./dataset/B01P00': ['1.tif',
  '10.tif',
  '11.tif',
  '12.tif',
  '13.tif',
  '14.tif',
  '15.tif',
  '16.tif',
  '17.tif',
  '18.tif',
  '19.tif',
  '2.tif',
  '20.tif',
  '21.tif',
  '22.tif',
  '23.tif',
  '24.tif',
  '25.tif',
  '26.tif',
  '27.tif',
  '28.tif',
  '29.tif',
  '3.tif',
  '30.tif',
  '31.tif',
  '32.tif',
  '33.tif',
  '34.tif',
  '35.tif',
  '36.tif',
  '37.tif',
  '38.tif',
  '39.tif',
  '4.tif',
  '40.tif',
  '41.tif',
  '42.tif',
  '43.tif',
  '44.tif',
  '45.tif',
  '46.tif',
  '47.tif',
  '48.tif',
  '49.tif',
  '5.tif',
  '50.tif',
  '51.tif',
  '52.tif',
  '53.tif',
  '54.tif',
  '55.tif',
  '56.tif',
  '57.tif',
  '58.tif',
  '59.tif',
  '6.tif',
  '60.tif',
  '61.tif',
  '62.tif',
  '63.tif',
  '64.tif',
  '65.tif',
  '7.tif',
  '8.tif',
  '9.tif'],
 './dataset/B02P00': ['1.tif',
  '10.tif',
  '11.tif',
  '12.tif',
  '13.tif',
  '14.tif',
  '15.tif',
  '16.tif',
  '17.tif',
  '18.tif',
  '19.tif',
  '2.tif',
  '20.tif',
  '21.tif',
  '22.tif',
  '23.tif'

In [71]:
subsets = [
    [1,7,8,9,36,37], #* subset 1 
    [2,5,10,11,12,13,15,39,40,41,42,44], #* subset 2 
    [3,6,14,16,17,19,20,43,45,46,48,49], #* subset 3 
    [18,21,22,23,24,25,26,47,50,51,52,53,54,55], #* subset 4 
    [4,27,28,29,30,31,32,33,34,35,56,57,58,59,60,61,62,63,64], #* subset 5 
]

In [72]:
# DataFrame from subsets and labels, and sort by index
subset_dataframe = pd.DataFrame([(index, label) for label in range(len(subsets))
                                    for index in subsets[label]], 
                                columns=['index', 'label'])\
                                .sort_values(by="index")
subset_dataframe 

,index,label
0,1,0
6,2,1
18,3,2
44,4,4
7,5,1
...,...,...
58,60,4
59,61,4
60,62,4
61,63,4


In [73]:
# DataFrame of image information with person assignments
data = []

for i, [container, filelist] in enumerate(images.items()):
   person_dir = container
   for j, filename in enumerate(filelist):
      data.append((j+1, i+1, os.path.join(*container.split("/"), filename)))

person_dataframe = pd.DataFrame(data, columns=['index', 'person', 'filepath'])

person_dataframe

,index,person,filepath
0,1,1,.\dataset\B01P00\1.tif
1,2,1,.\dataset\B01P00\10.tif
2,3,1,.\dataset\B01P00\11.tif
3,4,1,.\dataset\B01P00\12.tif
4,5,1,.\dataset\B01P00\13.tif
...,...,...,...
646,61,10,.\dataset\B10P00\64.tif
647,62,10,.\dataset\B10P00\65.tif
648,63,10,.\dataset\B10P00\7.tif
649,64,10,.\dataset\B10P00\8.tif


In [74]:
# Merge two DataFrames based on index
yalb_dataframe = pd.merge(person_dataframe, subset_dataframe, on="index")
yalb_dataframe  

,index,person,filepath,label
0,1,1,.\dataset\B01P00\1.tif,0
1,1,2,.\dataset\B02P00\1.tif,0
2,1,3,.\dataset\B03P00\1.tif,0
3,1,4,.\dataset\B04P00\1.tif,0
4,1,5,.\dataset\B05P00\1.tif,0
...,...,...,...,...
625,64,6,.\dataset\B06P00\8.tif,4
626,64,7,.\dataset\B07P00\8.tif,4
627,64,8,.\dataset\B08P00\7.tif,4
628,64,9,.\dataset\B09P00\8.tif,4


In [75]:
# Image Height and Width
img_w  = 160
img_h = 160

In [76]:
# load and preprocess an image
def load(path):
    img = preprocess.image.load_img(path, 
                                    grayscale=False, 
                                    color_mode="rgb", 
                                    target_size=(img_w, img_h))
    img_array = preprocess.image.img_to_array(img)/255
    return img_array

In [77]:
# Convolutional block class
class ConvBlock:
    def __init__(self, filters, kernel_size, strides, padding, pool_size, pool_strides, regularizer, pooling_type="max"):
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.pool_size = pool_size
        self.pool_strides = pool_strides
        self.regularizer = regularizer
        self.pooling_type = pooling_type

    def add_to_model(self, model):
        model.add(Conv2D(self.filters, self.kernel_size, strides=self.strides, padding=self.padding, kernel_regularizer=l2(self.regularizer)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        # Pooling layer
        if self.pool_size is not None:
            if self.pooling_type == "max":
                model.add(MaxPooling2D(pool_size=self.pool_size, strides=self.pool_strides))
            elif self.pooling_type == "avg": 
                model.add(AveragePooling2D(pool_size=self.pool_size, strides=self.pool_strides))
                
# Custom model class
class CustomModel:
    def __init__(self, input_shape, l2_regularization, initial_learning_rate, num_classes):
        self.model = Sequential()
        self.l2_regularization = l2_regularization
        self.initial_learning_rate = initial_learning_rate
        self.num_classes = num_classes
        self.input_shape = input_shape

    def build(self):
        # Add initial convolutional layer
        self.model.add(Conv2D(16, (3, 3), strides=(1, 1), input_shape=self.input_shape, padding='valid', kernel_regularizer=l2(self.l2_regularization)))
        # Convolutional blocks
        conv_blocks = [
            ConvBlock(16, (3, 3), (1, 1), 'valid', (4, 4), (4, 4), self.l2_regularization),
            ConvBlock(32, (3, 3), (1, 1), 'valid', (2, 2), (2, 2), self.l2_regularization),
            ConvBlock(64, (3, 3), (1, 1), 'valid', (2, 2), (2, 2), self.l2_regularization, pooling_type="avg")
        ]

        for block in conv_blocks:
            block.add_to_model(self.model)
        # Flattening, dense layers, and output
        self.model.add(Flatten())
        self.model.add(Dense(256, kernel_regularizer=l2(self.l2_regularization)))
        self.model.add(Activation('relu'))
        self.model.add(Dense(self.num_classes, activation='softmax'))

    """Compiles the model with optimizer, loss, and metrics"""
    def compile(self):
        self.model.compile(optimizer=Adam(learning_rate=self.initial_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


In [78]:
# Build, and compile the custom model
custom_model = CustomModel((img_w, img_h, 3), l2_regularization=hyper_parameters["l2-regularization"], initial_learning_rate=hyper_parameters["initial-learning-rate"], num_classes=2)
custom_model.build()
custom_model.compile()

In [79]:
# Define the optimizer, loss function
optimizer = Adam(hyper_parameters["initial-learning-rate"])

# Compile the model with the specified optimizer, loss function
custom_model.model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=["accuracy"])

In [80]:
# Data augmentation using ImageDataGenerator
datagen = preprocess.image.ImageDataGenerator(
    rotation_range=20,       # Random rotation between 0 and 20 degrees
    width_shift_range=0.2,   # Horizontal shift up to 20% of the width
    height_shift_range=0.2,  # Vertical shift up to 20% of the height
    shear_range=0.2,         # Shear transformations up to 20%
    zoom_range=0.2,          # Zooming up to 20%
    horizontal_flip=True,    # Horizontal flip
    vertical_flip=True,      # Vertical flip (disabled)
    )


In [81]:
# Sample two random cases
random_selected_cases = yalb_dataframe['person'].sample(2).tolist()

# Filter the DataFrame based on the selected cases
data = yalb_dataframe[yalb_dataframe['person'].isin(random_selected_cases)]

# Assign class labels
for idx, person in enumerate(random_selected_cases):
    data.loc[data['person'] == person, 'class'] = int(idx)

data 

C:\Users\V1TrAy\AppData\Local\Temp\ipykernel_23804\1494029414.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,index,person,filepath,label,class
2,1,3,.\dataset\B03P00\1.tif,0,0.0
4,1,5,.\dataset\B05P00\1.tif,0,1.0
12,2,3,.\dataset\B03P00\10.tif,1,0.0
14,2,5,.\dataset\B05P00\10.tif,1,1.0
22,3,3,.\dataset\B03P00\11.tif,2,0.0
...,...,...,...,...,...
604,62,5,.\dataset\B05P00\65.tif,4,1.0
612,63,3,.\dataset\B03P00\7.tif,4,0.0
614,63,5,.\dataset\B05P00\7.tif,4,1.0
622,64,3,.\dataset\B03P00\8.tif,4,0.0


In [82]:
#Split data into test and train sets based on label
test = data[data['label'] == 2]
train = data[data['label'] != 2]

#Test Set
test 

,index,person,filepath,label,class
22,3,3,.\dataset\B03P00\11.tif,2,0.0
24,3,5,.\dataset\B05P00\11.tif,2,1.0
52,6,3,.\dataset\B03P00\14.tif,2,0.0
54,6,5,.\dataset\B05P00\14.tif,2,1.0
132,14,3,.\dataset\B03P00\21.tif,2,0.0
134,14,5,.\dataset\B05P00\21.tif,2,1.0
152,16,3,.\dataset\B03P00\23.tif,2,0.0
154,16,5,.\dataset\B05P00\23.tif,2,1.0
162,17,3,.\dataset\B03P00\24.tif,2,0.0
164,17,5,.\dataset\B05P00\24.tif,2,1.0


In [83]:
#Train Set
train

,index,person,filepath,label,class
2,1,3,.\dataset\B03P00\1.tif,0,0.0
4,1,5,.\dataset\B05P00\1.tif,0,1.0
12,2,3,.\dataset\B03P00\10.tif,1,0.0
14,2,5,.\dataset\B05P00\10.tif,1,1.0
32,4,3,.\dataset\B03P00\12.tif,4,0.0
...,...,...,...,...,...
604,62,5,.\dataset\B05P00\65.tif,4,1.0
612,63,3,.\dataset\B03P00\7.tif,4,0.0
614,63,5,.\dataset\B05P00\7.tif,4,1.0
622,64,3,.\dataset\B03P00\8.tif,4,0.0


In [84]:
# Extract class labels and filepaths for training and testing
train_class         = np.array(train['class'])
test_class          = np.array(test['class'])

train_data          = np.array(train['filepath'])
test_data           = np.array(test['filepath'])

test_data, train_data

(array(['.\\dataset\\B03P00\\11.tif', '.\\dataset\\B05P00\\11.tif',
        '.\\dataset\\B03P00\\14.tif', '.\\dataset\\B05P00\\14.tif',
        '.\\dataset\\B03P00\\21.tif', '.\\dataset\\B05P00\\21.tif',
        '.\\dataset\\B03P00\\23.tif', '.\\dataset\\B05P00\\23.tif',
        '.\\dataset\\B03P00\\24.tif', '.\\dataset\\B05P00\\24.tif',
        '.\\dataset\\B03P00\\26.tif', '.\\dataset\\B05P00\\26.tif',
        '.\\dataset\\B03P00\\27.tif', '.\\dataset\\B05P00\\27.tif',
        '.\\dataset\\B03P00\\48.tif', '.\\dataset\\B05P00\\48.tif',
        '.\\dataset\\B03P00\\5.tif', '.\\dataset\\B05P00\\5.tif',
        '.\\dataset\\B03P00\\50.tif', '.\\dataset\\B05P00\\50.tif',
        '.\\dataset\\B03P00\\52.tif', '.\\dataset\\B05P00\\52.tif',
        '.\\dataset\\B03P00\\53.tif', '.\\dataset\\B05P00\\53.tif'],
       dtype=object),
 array(['.\\dataset\\B03P00\\1.tif', '.\\dataset\\B05P00\\1.tif',
        '.\\dataset\\B03P00\\10.tif', '.\\dataset\\B05P00\\10.tif',
        '.\\dataset\\B03P00\\

In [85]:
# Class for initializing and evaluating test/train data
class TestTrainInitializer: 
    def __init__(self, _data, _classes, loader=load) -> None:
        ch = map(lambda x: loader(x), _data)
        self.X_data = np.array([*ch])
        self.Y_data = kutils.to_categorical(_classes)

    def run(self):
        loss, accuracy = custom_model.model.evaluate(self.X_data, self.Y_data)
        return loss, accuracy

In [86]:
# Initialize TestTrainInitializer for training data
train_init = TestTrainInitializer(_data=train_data, _classes=train_class)
X_train, Y_train = train_init.X_data, train_init.Y_data 

In [87]:
# Extract data for analysis
people = np.array(random_selected_cases)
unique_labels_count = yalb_dataframe['label'].nunique()

In [88]:
# Data generator for training
train_generator = datagen.flow(X_train, 
                               Y_train, 
                               hyper_parameters["batch-size"])

In [89]:
# Progress bar callback for training
class TQDMProgressBar(Callback):
    def __init__(self, total):
        self.total = total
        self.pbar = None
        self.current_epoch = 0
        self.best_accuracy = float('-inf')  # Initialize with negative infinity
        self.worst_accuracy = float('inf')  # Initialize with positive infinity
        self.best_loss = float('inf')  # Initialize with positive infinity
        self.worst_loss = float('-inf')  # Initialize with negative infinity

    def on_epoch_begin(self, epoch, logs=None):
        self.current_epoch = epoch

    def on_epoch_end(self, epoch, logs=None):
        if self.current_epoch == epoch:
            self.pbar.update(1)
            current_accuracy = logs.get('accuracy', 0)
            current_loss = logs.get('loss', 0)
            
            # Update best accuracy
            if current_accuracy > self.best_accuracy:
                self.best_accuracy = current_accuracy
                print(f"\nNew Best Accuracy: {current_accuracy:.4f}")

            # Update worst accuracy
            if current_accuracy < self.worst_accuracy:
                self.worst_accuracy = current_accuracy
                print(f"New Worst Accuracy: {current_accuracy:.4f}", end='')

            # Update best loss
            if current_loss < self.best_loss:
                self.best_loss = current_loss
                print(f"\nNew Best Loss: {current_loss:.4f}", end='')

            # Update worst loss
            if current_loss > self.worst_loss:
                self.worst_loss = current_loss
                print(f"New Worst Loss: {current_loss:.4f}", end='')

    def on_train_begin(self, logs=None):
        self.pbar = tqdm(total=self.total, desc='Training', position=0, leave=True, dynamic_ncols=True)

    def on_train_end(self, logs=None):
        self.pbar.close()
        print(f"\nBest Accuracy: {self.best_accuracy:.4f}")
        print(f"Worst Accuracy: {self.worst_accuracy:.4f}")
        print(f"Best Loss: {self.best_loss:.4f}")
        print(f"Worst Loss: {self.worst_loss:.4f}")

# Create an instance of the TQDMProgressBar
progress_bar = TQDMProgressBar(total=hyper_parameters["number-of-epochs"])

# Train model with tqdm progress bar
train_model = custom_model.model.fit(train_generator,
                                     steps_per_epoch=int(len(X_train)/hyper_parameters["batch-size"]), 
                                     epochs=hyper_parameters["number-of-epochs"],
                                     callbacks=[progress_bar])


Training:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100
11/12 [==========================>...] - ETA: 0s - loss: 2.3146 - accuracy: 0.5698

Training:   1%|          | 1/100 [00:01<01:43,  1.04s/it]


New Best Accuracy: 0.5851
New Worst Accuracy: 0.5851
12/12 [==============================] - 1s 20ms/step - loss: 2.2214 - accuracy: 0.5851
Epoch 2/100
10/12 [========================>.....] - ETA: 0s - loss: 1.5505 - accuracy: 0.5385

Training:   2%|▏         | 2/100 [00:01<00:58,  1.69it/s]

...

12/12 [==============================] - 0s 20ms/step - loss: 0.4087 - accuracy: 0.9043
Epoch 100/100
10/12 [========================>.....] - ETA: 0s - loss: 0.3127 - accuracy: 0.9359

Training: 100%|██████████| 100/100 [00:28<00:00,  3.70it/s]


12/12 [==============================] - 0s 20ms/step - loss: 0.2907 - accuracy: 0.9468


Training: 100%|██████████| 100/100 [00:28<00:00,  3.52it/s]


Best Accuracy: 0.9574
Worst Accuracy: 0.5213
Best Loss: 0.2907
Worst Loss: 2.2214


In [90]:
# Visualize train loss to track model performance
train_losses = train_model.history['loss']
epochs = len(train_losses)

# Plot the results 
trace_train = go.Scatter(
    x=list(range(1, epochs + 1)),
    y=train_losses,
    mode='lines+markers',  
    name='Train Loss'
)

layout = go.Layout(
    title='Loss per Epoch (L/E)',
    xaxis=dict(title='Epoch'),
    yaxis=dict(title='Loss'),
    hovermode='closest'
)

fig = go.Figure(data=[trace_train], layout=layout)
pyo.iplot(fig)


In [91]:
# Initialize and run a test instance
test_1 = TestTrainInitializer(_data=test_data, _classes=test_class)
test_1_accuracy, test_1_loss = test_1.run()
test_1_accuracy, test_1_loss

1/1 [==============================] - 0s 120ms/step - loss: 0.1490 - accuracy: 1.0000


(0.14901798963546753, 1.0)

In [92]:
# Predict the test set
Y_test_pred = custom_model.model.predict(test_1.X_data)
predicted_labels = np.argmax(Y_test_pred, axis=1)

# Calculate precision, recall, and F1 score
performance_metrics = {
    'precision': scores.precision_score(test_class, predicted_labels, average='weighted'),
    'recall': scores.recall_score(test_class, predicted_labels, average='weighted'),
    'f1_score': scores.f1_score(test_class, predicted_labels, average='weighted')
}

# Accessing the scores
precision = performance_metrics['precision']
recall = performance_metrics['recall']
f1 = performance_metrics['f1_score']

1/1 [==============================] - 0s 73ms/step


In [93]:
precision, recall, f1 


(1.0, 1.0, 1.0)